# Creating a MISO2 config

In [1]:
import logging
import os
from datetime import datetime
from util.MISO2_file_manager import get_parameter_filenames
from model.config.MISO2_logger import get_MISO2_logger
from model.config.MISO2_config import MISO2Config
from preprocessing.checks.MISO2_input_data_sanity_check import input_data_sanity_check

Set file paths in relation to the directory where you are executing the code from

In [15]:
main_path = os.path.join(os.getcwd(), os.pardir, os.pardir, os.pardir)
input_path = os.path.join(main_path, "input", "input_usuk")
config_path = os.path.join(main_path, "config")
config_filename = 'MISO2_Config_USUK_v1.xlsx'
classification_filename = 'MISO2_Classifications_Master_USUK_v1.xlsx'
# config and classification need to be manually set. different configs may make use of the same set of input parameter files

Set logging levels. Use a level lower than WARN only for debugging purposes 

In [16]:
timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")

get_MISO2_logger(
    log_filename='MISO2_log_' + timestamp + ".txt",
    log_pathname=os.path.join(main_path, 'logs'),
    file_level=None,
    console_level=logging.WARN)


(<RootLogger root (WARNING)>, <StreamHandler stderr (WARNING)>, None)

It never hurts to make a sanity data check of the input files. Note that this has a write back option available which you probably do not want to use blindly. The sanity check controls for dimensions and data availability and emits warnings if it encounters a mismatch. Output is omitted here.

In [17]:
filenames = get_parameter_filenames(input_path, config_filename)
input_data_sanity_check(input_path, filenames, write_back=False)

2025-03-03 17:36:32,035 WARNING (MISO2_input_data_sanity_check.py <check_sheet_size>): For c:\Users\simsam\Desktop\GitHub\MISO2\docs\source\notebooks\..\..\..\input\input_usuk\MISO2_EndUseShares_v1.xlsx Data values cols do not match cover sheet: 197 vs 201
2025-03-03 17:36:32,035 WARNING (MISO2_input_data_sanity_check.py <check_sheet_size>): WARNING: Number of cols in cover sheet is smaller than number of data cols,                       some values will not be parsed
2025-03-03 17:36:32,185 WARNING (MISO2_input_data_sanity_check.py <check_values>): Input data parameter c:\Users\simsam\Desktop\GitHub\MISO2\docs\source\notebooks\..\..\..\input\input_usuk\MISO2_EoLRateRecycling_v1.xlsx misses values of type Material which are present in classification file: {'cement', 'bitumen'} 

2025-03-03 17:36:32,185 WARNING (MISO2_input_data_sanity_check.py <check_values>): Input data parameter c:\Users\simsam\Desktop\GitHub\MISO2\docs\source\notebooks\..\..\..\input\input_usuk\MISO2_EoLRateDowncycl

Parsing uses ODYM functionality

In [18]:
miso_config = MISO2Config(data_path=input_path,
                          config_path=config_path,
                          timestamp=timestamp)

miso_config.initialise_from_excel(config_filename=config_filename,
                                  classification_filename=classification_filename,
                                  parse_uncertainty=True)

2025-03-03 17:36:35,926 CRITICAL (ODYM_Functions.py <ReadParameterXLSX>): CLASSIFICATION FILE FATAL ERROR: Classification file of parameter MISO2_Lifetimes is not identical to the classification master file used for the current model run.
2025-03-03 17:36:37,801 CRITICAL (ODYM_Functions.py <ReadParameterXLSX>): CLASSIFICATION FILE FATAL ERROR: Classification file of parameter MISO2_EoLRateRecycling is not identical to the classification master file used for the current model run.
2025-03-03 17:36:38,353 CRITICAL (ODYM_Functions.py <ReadParameterXLSX>): CLASSIFICATION FILE FATAL ERROR: Classification file of parameter MISO2_EoLRateDowncycling is not identical to the classification master file used for the current model run.
2025-03-03 17:36:38,677 CRITICAL (ODYM_Functions.py <ReadParameterXLSX>): CLASSIFICATION FILE FATAL ERROR: Classification file of parameter MISO2_WasteRate_recov_p5 is not identical to the classification master file used for the current model run.
2025-03-03 17:36:39

Save the config for later use in a binary format

In [21]:
# This whole code was additionally implemented by Simon Graf on the 03.03.2025 since there was a FileNotFoundError for the pickle saving
# Define the folder path where you want to save the pickle
folder_path = os.path.join(main_path, "config")

# Create the folder if it doesn't exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print("Folder created!")
else:
    print("This folder already exists.")

# Now save the pickle
miso_config.save_to_pickle("selected_countries_config.pickle")

This folder already exists.


In [20]:
# Get the current working directory
current_folder = os.getcwd()

# Save the pickle in the current folder
miso_config.save_to_pickle("selected_countries_config.pickle", folder=current_folder)

In [ ]:
#miso_config.save_to_pickle("selected_countries_config.pickle") #thats the original code. However, this saves the pickle into current directionary, instead of a config folder in main, where it is needed for later processing.
